# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

In [5]:
from cassandra.cluster import Cluster
import csv
import pandas as pd

## Base CSV
The file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory, contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

### Load the base csv

In [6]:
file = 'event_datafile_new.csv'

## Target Queries

The queries are stated as follows:

##### Query 1

> Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

##### Query 2

> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

##### Query 3

> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Query Implementation

The queries are translated into CQL format and appropriate names for the tables are chosen:

In [7]:
name_1 = "session_history"
name_2 = "user_history"
name_3 = "song_listening_users"

In [8]:
query_1 = "SELECT artist, song, length FROM " + name_1 + " WHERE session_id=338 AND item_in_session=4"
query_2 = "SELECT artist, song, first_name, last_name FROM " + name_2 + " WHERE session_id=182 AND user_id=10"
query_3 = "SELECT first_name, last_name FROM " + name_3 + " WHERE song='All Hands Against His Own'"

# Definition of Helper Functions

We define a number of helper functions to streamline the ETL pipeline:

In [9]:
def try_execute(session, query):
    '''Safely execute the query.'''
    try:
        session.execute(query)
    except Exception as e:
        print(e)
        
def delete_table(session, table_name):
    '''Drop the table with the given name.'''
    try_execute(session, "DROP TABLE IF EXISTS " + table_name + ";")
    
def create_table(session, table_name, fields):
    '''Create the table with the given name and fields.'''
    try_execute(session, "CREATE TABLE " + table_name + " " + fields + ";")
    
def insert(session, table_name, fields, values):
    '''Insert values into the table.'''
    try:
        session.execute("INSERT INTO " + table_name + " " + fields, values)
    except Exception as e:
        print(e)
        
def test_select(session, query):
    '''Test the query and return a pandas dataframe'''
    def pandas_factory(colnames, rows):
        return pd.DataFrame(rows, columns=colnames)

    session.row_factory = pandas_factory
    session.default_fetch_size = None
    try:
        rows = session.execute(query)
        return rows._current_rows
    except Exception as e:
        print(e)

## Cluster and Keyspace Setup

#### Creating a Cluster

In [10]:
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)


#### Creating a Keyspace

In [11]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Seting the Keyspace

In [12]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## ETL core

### Delete old tables (if necessary)

In [13]:
delete_table(session, name_1)
delete_table(session, name_2)
delete_table(session, name_3)

### Create tables with required fields

- The second query implies that only sessionId, userId and itemInSession combined form a unique primary key; but not necessarily any combination of two out of these three fields
- The suffix "IF NOT EXISTS" was added to the insertion statements to avoid conflicts through duplicates
- The columns were arranged as: 1) partition key, 2) clustering keys, 3) desired fields in the order of the query

In [14]:
# The user_id is not required for the first query, but necessary to create a unique primary key. 
# The columns are then composed of the fields in the primary key and the desired fields for the query.  
table_1_fields = "(session_id int, item_in_session int, user_id int, artist text, song text, length float, " \
    + "PRIMARY KEY (session_id, item_in_session, user_id))"
table_1_inserts = "(session_id, item_in_session, user_id, artist, song, length) VALUES (%s, %s, %s, %s, %s, %s) IF NOT EXISTS"

# The second query can be treated analogues to the first. Unfortunately, it is not possible to create a single table
# which covers both query 1 and query 2, since the relevant keys are incompatible.
table_2_fields = "(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, "\
    + "PRIMARY KEY (user_id, session_id, item_in_session))"
table_2_inserts = "(user_id, session_id, item_in_session, artist, song, first_name, last_name) VALUES (%s, %s, %s, %s, %s, %s, %s) IF NOT EXISTS"

# For the third query, we can disregard most fields, as all that matters is which users ever listened to a given song at least once.
# We can achieve this by simply ignoring duplicates when inserting data into the table.
# The scenario where two different users by the same name listened to the given song is not covered by the query.
# In order to properly treat this case, one would need to include the user_id in the query and the table. 
table_3_fields = "(song text, first_name text, last_name text, "\
    + "PRIMARY KEY (song, last_name, first_name))" 
table_3_inserts = "(song, first_name, last_name) VALUES (%s, %s, %s) IF NOT EXISTS"

In [15]:
create_table(session, name_1, table_1_fields)
create_table(session, name_2, table_2_fields)
create_table(session, name_3, table_3_fields)

### Insert csv data

In [16]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    i = 0
    for line in csvreader:
        insert(session, name_1, table_1_inserts, (int(line[8]), int(line[3]), int(line[10]), line[0], line[9], float(line[5])))
        insert(session, name_2, table_2_inserts, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        insert(session, name_3, table_3_inserts, (line[9], line[1], line[4]))
        
        i += 1
        if i%1000 == 0:
            print("Processed " + str(i) + " lines;")
    print("Done.")

Processed 1000 lines;
Processed 2000 lines;
Processed 3000 lines;
Processed 4000 lines;
Processed 5000 lines;
Processed 6000 lines;
Done.


## Query testing

#### Query 1

In [17]:
test_select(session, query_1)

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


#### Query 2

In [18]:
test_select(session, query_2)

,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### Query 3

In [19]:
test_select(session, query_3)

,first_name,last_name
0,Sara,Johnson
1,Tegan,Levine
2,Jacqueline,Lynch


## Drop the tables and close out the sessions

In [20]:
delete_table(session, name_1)
delete_table(session, name_2)
delete_table(session, name_3)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()